In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False
torch.set_num_threads(8)

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',
            # 'model': 'MLP',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 15,
            'per_task_examples': np.inf,
            # 'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            # 'criterion': torch.nn.BCEWithLogitsLoss(),

            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.00,
            'metric' : "EO",
            'lambda': 1.0,
            'lambda_old': 0.0,
            'solver': "absolute_minsum_LP_solver_v3",
            'converter': "converter_LP_absolute_additional_EO",
            
              }
    

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lmbd_{params['lambda']}_lmbdold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=BiasedMNIST/seed=10_epoch=15_lr=0.001_tau=10.0_alpha=0.0_lmbd_1.0_lmbdold_0.0


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                random_class_idx=False)
    input_dim = (3, 28, 28)
    class_idx = benchmark.class_idx
    num_classes = len(class_idx)

[0 1 2 3 4 5 6 7 8 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2][sample_idx][0]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img, target, *_ in benchmark.tests[incremental_step]][20:30], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer2
from metrics import FairMetricCollector
from metrics import MetricCollector2

from algorithms import Heuristic3
from algorithms.fairl import FaIRL
from algorithms.icarl import iCaRL
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
# backbone = ResNet18Small2(
#         input_dim=input_dim, 
#         output_dim=num_classes,
#         class_idx=class_idx,
#         config=params
#     ).to(params['device'])
# algorithm = iCaRL(backbone, benchmark, params, requires_memory=True)
algorithm = Heuristic3(backbone, benchmark, params, requires_memory=True)

metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])
# metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
#                                                         eval_interval='epoch',
#                                                         epochs_per_task=params['epochs_per_task'])
from trainers.baselines import BaseMemoryContinualTrainer as ContinualTrainer

# trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])

trainer = FairContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

fairness_metrics = ["std", "EER", "EO", "DP"]
for metric in metric_manager_callback.meters:
    if metric in fairness_metrics:
        metric_manager_callback.meters[metric].agg = agg


In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.850024723545806, 'loss': 0.0019389718700526172, 'EO': [0.09444444444444444, 0.22201834862385322], 'DP': -1, 'accuracy_s0': 1.0, 'accuracy_s1': 0.6835372069317023, 'classwise_accuracy': {0: array([895, 980]), 1: array([ 893, 1135])}, 'DP_ingredients': {'class_pred_count_s0': {0: 530, 1: 590}, 'class_pred_count_s1': {0: 607, 1: 388}, 'class_pred_count': {0: 1137, 1: 978}, 'count_s0': 1120, 'count_s1': 995, 'count': 2115}}
[2] Eval metrics for task 1 >> {'accuracy': 0.9226018160568192, 'loss': 0.0010324257626319327, 'EO': [0.03999999999999998, 0.12293577981651377], 'DP': -1, 'accuracy_s0': 1.0, 'accuracy_s1': 0.8370642201834863, 'classwise_accuracy': {1: array([1001, 1135]), 0: array([944, 980])}, 'DP_ingredients': {'class_pred_count_s0': {1: 590, 0: 530}, 'class_pred_count_s1': {1: 447, 0: 548}, 'class_pred_count': {1: 1037, 0: 1078}, 'count_s0': 1120, 'count_s1': 995, 'count': 2115}

In [9]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.997, 0.   , 0.   , 0.   , 0.   ],
       [0.969, 0.939, 0.   , 0.   , 0.   ],
       [0.955, 0.839, 0.928, 0.   , 0.   ],
       [0.916, 0.801, 0.86 , 0.96 , 0.   ],
       [0.87 , 0.745, 0.735, 0.897, 0.892]])

In [10]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

0.9140821888880352

In [11]:
[np.round(x, 3) for x in metric_manager_callback.meters['EO'].compute_overall()]

[0.003, 0.039, 0.067, 0.071, 0.112]

In [12]:
np.mean(metric_manager_callback.meters['EO'].compute_overall())

0.05839937723842418

In [13]:
[np.round(x, 3) for x in metric_manager_callback.meters['DP'].compute_overall()]

[0.008, 0.006, 0.007, 0.01, 0.012]

In [14]:
np.mean(metric_manager_callback.meters['DP'].compute_overall())

0.008653813883898914

In [15]:
task_id = 2

print(f"{task_id=}")
print(f"sensitive samples / all samples = {(benchmark.trains[task_id].sensitives != benchmark.trains[task_id].targets).sum().item()} / {benchmark.trains[task_id].sensitives.shape[0]}")

updated_seq_indices = benchmark.seq_indices_train[task_id]
print(f"sensitive samples / selected samples = {(benchmark.trains[task_id].sensitives[updated_seq_indices] != benchmark.trains[task_id].targets[updated_seq_indices]).sum().item()} / {len(updated_seq_indices)}")


task_id=2
sensitive samples / all samples = 604 / 12089
sensitive samples / selected samples = 604 / 12089


In [16]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 5958 --> 5958
3 : 6131 --> 6131
incremental_step=3
4 : 5842 --> 5842
5 : 5421 --> 5421
incremental_step=4
6 : 5918 --> 5918
7 : 6265 --> 6265
incremental_step=5
8 : 5851 --> 5851
9 : 5949 --> 5949


In [17]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{1: array([1130, 1135]), 0: array([979, 980])},
 {1: array([1110, 1135]),
  0: array([940, 980]),
  2: array([ 967, 1032]),
  3: array([ 950, 1010])},
 {0: array([918, 980]),
  1: array([1104, 1135]),
  3: array([ 840, 1010]),
  2: array([ 874, 1032]),
  5: array([802, 892]),
  4: array([940, 982])},
 {1: array([1065, 1135]),
  0: array([875, 980]),
  2: array([ 801, 1032]),
  3: array([ 835, 1010]),
  4: array([878, 982]),
  5: array([737, 892]),
  6: array([922, 958]),
  7: array([ 985, 1028])},
 {0: array([823, 980]),
  1: array([1022, 1135]),
  3: array([ 724, 1010]),
  2: array([ 797, 1032]),
  4: array([705, 982]),
  5: array([670, 892]),
  6: array([877, 958]),
  7: array([ 903, 1028]),
  9: array([ 864, 1009]),
  8: array([904, 974])}]